In [4]:
require 'countries'
require 'numo/narray'
require 'nyaplot'

false

In [11]:
def sim(v1, v2)
  v1.transpose.dot(v2) / Math.sqrt(v1.transpose.dot(v1) * v2.transpose.dot(v2))
end

def distance(mat, v1)
  result = Hash.new
  mat.shape[0].times do |i|
    v2 = mat[i, true]
    similarity = sim(v1, v2)
    result[i] = similarity
  end

  result.sort{|(_, val1), (_, val2)| val2 <=> val1}[1..10]
end

def analogy(mat, v1, v2, v3)
  result = Hash.new
  v4 = v1 - v2 + v3

  mat.shape[0].times do |i|
    v_ = mat[i, true]
    similarity = sim(v4, v_)
    result[i] = similarity
  end

  result.sort{|(_, val1), (_, val2)| val2 <=> val1}[1..10]
end

:analogy

# 90. word2vecによる学習

81で作成したコーパスに対してword2vecを適用し，単語ベクトルを学習せよ．さらに，学習した単語ベクトルの形式を変換し，86-89のプログラムを動かせ．

In [12]:
unless File.exists?('../data/knock90')
  corpus = Marshal.load(open('../data/knock81.dump'))
  output = open('../data/enwiki_corpus.txt', 'w')
  corpus.each do |document|
    output.puts document
  end
  output.close

  puts 'NOTE: after running this script, please execute following command to create embedding model.'
  puts 'word2vec  -train ../../data/enwiki_corpus.txt -output ../../data/knock90'
  puts 'thanks !'

else

  token2id = Hash.new
  id2token = Hash.new
  mat = nil

  File.foreach('../data/knock90').with_index do |line, index|
    elements = line.chomp.split

    if index == 0
      i, j = elements.map(&:to_i)
      mat = Numo::Float64.zeros(i, j)
      next
    end

    id = index - 1
    token = elements[0]
    vector = elements[1..-1].map(&:to_f)

    token2id[token] = id
    id2token[id] = token

    vector.each_with_index do |xy, d|
      mat[id, d] = xy
    end
  end

end

1

Errno::ENOENT: No such file or directory @ rb_sysopen - ../data/knock81.dump

In [ ]:
word_matrix = Numo::Float64.zeros(mat.size, mat[0].size)
mat.each_with_index do |vector, i|
  vector.each_with_index do |x, j|
    word_matrix[i, j] = x
  end
end


vec1 = word_matrix[token2id['United_States'], true]
vec2 = word_matrix[token2id['US'], true]

p 'United_States'
p token2id['United_States']
p vec1
p 'US'
p token2id['US']
p vec2



mat.each_with_index do |vector, i|
  vector.each_with_index do |x, j|
    word_matrix[i, j] = x
  end
end

vec1 = word_matrix[token2id['United_States'], true]
vec2 = word_matrix[token2id['US'], true]

p sim(vec1, vec2)

vec1 = word_matrix[token2id['England'], true]
distance(word_matrix, vec1).map{|id, sim| [id2token[id], sim]}.each do |token, sim|
  puts "#{token}\t#{sim}"
end


vec1 = word_matrix[token2id['Spain'], true]
vec2 = word_matrix[token2id['Madrid'], true]
vec3 = word_matrix[token2id['Athens'], true]

analogy(word_matrix, vec1, vec2, vec3).map{|id, sim| [id2token[id], sim]}.each do |token, sim|
  puts "#{token}\t#{sim}"
end

## 注意

81でつくったdocをテキストファイルに出力したやつ(../data/enwiki_for_knock81.txt)を使う

```ruby
doc.each do |sentence|
  puts sentence
end
```

しただけ．

# 91. アナロジーデータの準備

単語アナロジーの評価データをダウンロードせよ．このデータ中で": "で始まる行はセクション名を表す．例えば，": capital-common-countries"という行は，"capital-common-countries"というセクションの開始を表している．ダウンロードした評価データの中で，"family"というセクションに含まれる評価事例を抜き出してファイルに保存せよ．

In [ ]:
problems = []
writable = false

File.foreach('../../data/questions-words.txt').map(&:chomp).each do |line|
  if line == ": family"
    writable = true
    next
  end

  if line =~ /^\:/ && writable
    break
  end

  problems << line.split if writable
end

Marshal.dump(problems, open('../../data/knock91.dump', 'wb'))


# 92. アナロジーデータへの適用

91で作成した評価データの各事例に対して，vec(2列目の単語) - vec(1列目の単語) + vec(3列目の単語)を計算し，そのベクトルと類似度が最も高い単語と，その類似度を求めよ．求めた単語と類似度は，各事例の末尾に追記せよ．このプログラムを85で作成した単語ベクトル，90で作成した単語ベクトルに対して適用せよ．

In [ ]:
def apply(word_matrix, token2id, id2token, problems)
  results = []

  problems.each do |problem|
    next unless token2id[problem[0]] && token2id[problem[1]] && token2id[problem[2]] && token2id[problem[3]]
    vec1 = word_matrix[token2id[problem[0]], true]
    vec2 = word_matrix[token2id[problem[1]], true]
    vec3 = word_matrix[token2id[problem[2]], true]
    vec4 = word_matrix[token2id[problem[3]], true]

    most_similar_id = analogy(word_matrix, vec1, vec2, vec3)[0][0]
    vec4_ = word_matrix[most_similar_id, true]
    similarity = sim(vec4, vec4_)

    puts "#{problem[3]}\t#{id2token[most_similar_id]}\t#{similarity}"
    results << [problem[3], id2token[most_similar_id], similarity]
  end

  results
end


puts "applying start... (it takes few minitutes)"
results = apply(word_matrix, token2id, id2token, problems)

# 93. アナロジータスクの正解率の計算

92で作ったデータを用い，各モデルのアナロジータスクの正解率を求めよ．

In [ ]:
result_sims = results.map{|a, b, c| c}
puts "score: #{result_sims.select{|c| c == 1}.size.to_f / result_sims.size}"

# 94. WordSimilarity-353での類似度計算

The WordSimilarity-353 Test Collectionの評価データを入力とし，1列目と2列目の単語の類似度を計算し，各行の末尾に類似度の値を追加するプログラムを作成せよ．このプログラムを85で作成した単語ベクトル，90で作成した単語ベクトルに対して適用せよ．

In [ ]:
results = []
File.foreach('../../data/combined.tsv') do |line|
  word1, word2, score = line.split
  score = score.to_f
  
  next unless token2id[word1] && token2id[word2]
  similarity = sim(word_matrix[token2id[word1], true], word_matrix[token2id[word2], true])
  puts "#{word1}\t#{word2}\t#{similarity}\t#{score}"
  results << [word1, word2, similarity, score]
end

# 95. WordSimilarity-353での評価

94で作ったデータを用い，各モデルが出力する類似度のランキングと，人間の類似度判定のランキングの間のスピアマン相関係数を計算せよ．

保留

# 96. 国名に関するベクトルの抽出

word2vecの学習結果から，国名に関するベクトルのみを抜き出せ．

In [ ]:
countries.each do |country|
  if token2id[country]
    p country
    p word_matrix[token2id[country], true]
  end
end

# 97. k-meansクラスタリング

96の単語ベクトルに対して，k-meansクラスタリングをクラスタ数k=5k=5として実行せよ．

In [ ]:
# NOTE: we can get data more simply (now, I do not use this procedure because k-means needs to be fixed)
# data = countries.map {|country| token2id[country] ? mat[token2id[country]] : nil }.select {|arr| arr}
# mat = Numo::Float64[*data]

vectors = []
labels = []
num_class = []

countries.each do |country|
  if token2id[country]
    labels << country
    vectors << word_matrix[token2id[country], true]
    num_class << 0
  end
end

# k-means
#
# parameter
k = 5

# initialize
centroids = vectors.shuffle.first(k)

10.times do |num_iter|
  puts "iter: #{num_iter}"

  # assign phase
  vectors.each_with_index do |vector, i|
    similarities = centroids.map do |centroid|
      sim(vector, centroid)
    end

    max_sim = similarities.max
    argmax = similarities.index(max_sim)
    num_class[i] = argmax
  end

  # maximization
  centroids.size.times do |t|
    new_centroid = Numo::Float64.zeros(centroids[0].shape)

    num_points = 0
    vectors.each_with_index do |vector, i|
      if num_class[i] == t
        new_centroid += vector 
        num_points += 1
      end
    end

    centroids[t] = new_centroid / num_points
  end
end

puts

centroids.size.times do |i|
  puts "cluster_#{i}"

  cluster = []
  vectors.size.times do |t|
    cluster << labels[t] if num_class[t] == i
  end

  puts cluster.sort.join(' ')
  puts
end

# 98. Ward法によるクラスタリング
96の単語ベクトルに対して，Ward法による階層型クラスタリングを実行せよ．さらに，クラスタリング結果をデンドログラムとして可視化せよ．

# 99. t-SNEによる可視化
96の単語ベクトルに対して，ベクトル空間をt-SNEで可視化せよ．

In [3]:
countries = ISO3166::Country.all.map{|obj| obj.name.gsub(/\s/, '_')}
token2id, _, mat = Marshal.load(open('../data/knock90.dump', 'rb'))

data = countries.map {|country| token2id[country] ? mat[token2id[country]] : nil }.select {|arr| arr}
names = countries.map {|country| token2id[country] ? country : nil}.select {|arr| arr}
mat = Numo::Float64[*data]


# TODO: random initialize should be done with normal distribution
tsne_mat = Numo::Float64.new(mat.shape[0], 2).rand

# perp = 0.1
num_iter = 10
eta = 0.01
# momentum = alpha

# compute p_{i, j}
p = Numo::Float64.zeros(mat.shape[0], mat.shape[0])
q = Numo::Float64.zeros(mat.shape[0], mat.shape[0])

# I compute denominator here because denominator of p_{i, j} (appeared in Equation (3)) is constant when X are given.
omega = 1 # TODO: need to compute a variance
denominator = 0
tsne_mat.shape[0].size.times do |k|
  tsne_mat.shape[0].size.times do |l|
    next if k == l
    elem = mat[k, true] - mat[l, true]
    denominator += Math.exp(- elem.transpose.dot(elem) / omega)
  end
end


tsne_mat.shape[0].times do |i|
  tsne_mat.shape[0].times do |j|
    elem1 = mat[i, true] - mat[j, true]
    numerator = Math.exp(- elem1.transpose.dot(elem1) / omega)
    p_j_i = numerator / denominator
    elem2 = mat[j, true] - mat[j, true]
    numerator = Math.exp(- elem2.transpose.dot(elem2) / omega)
    p_i_j = numerator / denominator
    p[i, j] = p_i_j + p_j_i
  end
end


num_iter.times do |t|
  denominator = 0
  tsne_mat.shape[0].size.times do |k|
    tsne_mat.shape[0].size.times do |l|
      next if k == l
      elem = tsne_mat[k, true] - tsne_mat[l, true]
      denominator += 1 + elem.transpose.dot(elem)
    end
  end

  tsne_mat.shape[0].size.times do |i|
    tsne_mat.shape[0].size.times do |j|
      # compute low-dimensional affinities q_{i, j} using Equation 4
      elem1 = tsne_mat[i, true] - tsne_mat[j, true]
      numerator = 1 + elem1.transpose.dot(elem1)

      # NOTE: numerator^{-1} / denominator^{-1}
      q[i, j] = denominator / numerator
    end
  end

  # compute gradient (using Equation 5)
  gradient = Numo::Float64.zeros(tsne_mat.shape)
  tsne_mat.shape[0].times do |i|
    # numerator = 0
    tsne_mat.shape[0].times do |j|
      elem1 = p[i, j] - q[i, j] # scalar
      elem2 = (tsne_mat[i, true] - tsne_mat[j, true]) # vector
      elem3 = 1 / (1 + elem2.transpose.dot(elem2)) # scalar

      dcdy_i = 4 * elem1 * elem2 / elem3

      # TODO: use for loop
      gradient[i, 0] = dcdy_i[0]
      gradient[i, 1] = dcdy_i[1]
    end
  end

  # update
  tsne_mat = tsne_mat + eta * gradient
end


x = tsne_mat[true, 0]
y = tsne_mat[true, 1]

plt = Nyaplot::Plot.new
plt.add(:scatter, x, y)
plt.show()

Errno::ENOENT: No such file or directory @ rb_sysopen - ../data/knock90.dump